# Treinamento e Avaliação de uma CNN (VGG16) com CIFAR10
Este notebook replica o fluxo do script `main_cnn.py`, realizando o treinamento e avaliação de uma rede neural convolucional (CNN) baseada na arquitetura VGG16 utilizando o dataset CIFAR10.

Cada etapa do processo está separada em células, com explicações em português.

## 1. Importação de Bibliotecas
Importe as bibliotecas necessárias para o treinamento, avaliação e manipulação dos dados.

In [1]:
import torch
from torch import nn, optim

# Carregamento de Dados e Modelos
from torch.utils.data import DataLoader
from torchvision import datasets, models
from torchvision import transforms

# Plots e análises
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import time, os

from cnn.modelo import Modelo
from utils.imagem_utils import ImagemUtils
from utils.metricas import Metricas

import logging
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

## 2. Definição de Parâmetros e Dispositivo
Defina os hiperparâmetros do treinamento e selecione o dispositivo (GPU ou CPU) conforme disponibilidade.

In [2]:
# Definição dos hiperparâmetros e dispositivo
args = {
    'num_epocas': 1,     # Número de épocas de treinamento
    'taxa_aprendizado': 1e-3,           # Taxa de aprendizado
    'penalidade': 8e-4, # Penalidade (weight decay)
    'tamanho_lote': 10, # Tamanho do lote
    'qtd_classes': 10   # Número de classes do CIFAR10
}

if torch.cuda.is_available():
    args['dispositivo'] = torch.device('cuda')
    logging.info("CUDA disponível. Usando GPU para treinamento.")
else:
    args['dispositivo'] = torch.device('cpu')
    logging.info("CUDA não disponível. Usando CPU para treinamento.")

2025-09-28 11:50:41,646 - INFO - CUDA não disponível. Usando CPU para treinamento.


## 3. Definição das Transformações de Imagem
Utilize as transformações recomendadas para o dataset, baseadas no ImageNet.

In [3]:
# Definindo transformações para imagens (ImageNet)
transform = ImagemUtils.opcoes_transformacao_imagenet()
logging.info("Transformações de imagem definidas.")

2025-09-28 11:50:41,665 - INFO - Transformações de imagem definidas.


## 4. Carregamento dos Dados CIFAR10
Carregue os conjuntos de treino e teste do CIFAR10, aplicando as transformações definidas.

In [4]:
# Carregando conjunto de treino e teste CIFAR10
train_set = datasets.CIFAR10('.',
                      train=True,
                      transform=transform,
                      download=True)

logging.info("Conjunto de treino carregado.")

test_set = datasets.CIFAR10('.',
                      train=False,
                      transform=transform,
                      download=False)
logging.info("Conjunto de teste carregado.")

2025-09-28 11:50:43,815 - INFO - Conjunto de treino carregado.
2025-09-28 11:50:44,239 - INFO - Conjunto de teste carregado.


## 5. Criação dos DataLoaders
Crie os DataLoaders para os conjuntos de treino e teste, utilizando o tamanho de lote definido e embaralhamento dos dados.

In [5]:
# Criação dos DataLoaders
train_loader = DataLoader(train_set,
                          batch_size=args['tamanho_lote'],
                          shuffle=True)
test_loader = DataLoader(test_set,
                          batch_size=args['tamanho_lote'],
                          shuffle=True)

logging.info(f"Tamanho do lote: {args['tamanho_lote']}")
logging.info(f"Número de classes: {args['qtd_classes']}")

2025-09-28 11:50:44,259 - INFO - Tamanho do lote: 10
2025-09-28 11:50:44,262 - INFO - Número de classes: 10


## 6. Inicialização da Função de Perda
Inicialize a função de perda CrossEntropyLoss e mova para o dispositivo selecionado.

In [6]:
# Inicialização da função de perda
criterio = nn.CrossEntropyLoss().to(args['dispositivo'])
logging.info("Função de perda inicializada.")

2025-09-28 11:50:44,284 - INFO - Função de perda inicializada.


## 7. Inicialização do Modelo CNN
Inicialize o modelo CNN (VGG16) usando a classe Modelo, passando os DataLoaders, parâmetros e função de perda.

In [7]:
# Inicialização do modelo CNN (VGG16)
modelo = Modelo(
    train_loader=train_loader,
    test_loader=test_loader,
    args=args,
    criterio=criterio
)
modelo.iniciar_modelo_resnet18()
logging.info("Modelo CNN inicializado.")

/home/marinapiragibe/tcc-uff/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/marinapiragibe/tcc-uff/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


2025-09-28 11:50:45,008 - INFO - Modelo CNN inicializado.


## 8. Treinamento do Modelo
Execute o método `treinar()` do modelo para realizar o treinamento da CNN.

In [8]:
# Treinamento do modelo CNN
logging.info("Iniciando treinamento do modelo CNN")
modelo.treinar()
logging.info("Treinamento concluído!")

2025-09-28 11:50:45,033 - INFO - Iniciando treinamento do modelo CNN


332/5000

KeyboardInterrupt: 

## 9. Teste do Modelo
Execute o método `testar()` do modelo para obter as classes preditas e reais no conjunto de teste.

In [ ]:
# Teste do modelo CNN
logging.info("Iniciando teste do modelo CNN")
classes_preditas, classes_reais = modelo.testar()
logging.info("Teste concluído!")

## 10. Cálculo e Exibição das Métricas
Utilize a classe `Metricas` para calcular e exibir as métricas de desempenho do modelo.

In [ ]:
# Cálculo e exibição das métricas de desempenho
metricas = Metricas(classes_reais=classes_reais, classes_preditas=classes_preditas)
logging.info("Calculando métricas de desempenho")
metricas.calcular_e_imprimir_metricas()